In [25]:
import pandas as pd
from datetime import datetime, timedelta, date
import pandas.io.sql as psql
import pandasql
from sqlalchemy import create_engine, MetaData
from sqlalchemy.sql import text as sa_text
import win32com.client
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#Function for chunking lists
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

#Define Connection to Postgres Database
def connect(user, password, db, host, port=5432):
    url  = 'postgresql://{}:{}@{}:{}/{}'
    url = url.format(user, password, host, port, db)

    # The return value of create_engine() is our connection object
    con = create_engine(url, client_encoding='utf8')
    return con

def migrate_records(dataframe, destination_table, connection_object):
    dataframe.to_sql(destination_table, connection_object, if_exists='append', index=False)
    


In [4]:
#Connection criteria for postgresdb    
hostname = 'mktstrategy.ciklurvi0auw.us-east-1.rds.amazonaws.com'
username = 'tronc'
password = 'tronc123123!'
database = 'Crediting'

#create connection object
c = connect(username, password, database, hostname)

#Define OLAP cube connection string and destination table to ingest
connString = "PROVIDER=MSOLAP;Data Source={0};Database={1}".format('fcwPsqlanl03','TMModelingCube')
destination_table = 'sales_assignment_analysis'

#truncate existing table
#c.execute(sa_text('''TRUNCATE TABLE "billing"''').execution_options(autocommit=True))

In [40]:
#Bring in all SA Ids From Crediting DB
rep_mappings = pd.read_sql('SELECT DISTINCT company_name, sa_id, rep_name, manager_name, fiscal_year, sales_team FROM weekly_progress_report', c)
sa_id_list = list(rep_mappings.sa_id.unique())
sa_id_list = str(list(map(lambda x: '[TM Sales Assignment].[SA ID].&['+x+']',sa_id_list)))[1:-1].replace("'","")

#Get Customer Name Numbers
connString = "PROVIDER=MSOLAP;Data Source={0};Database={1}".format('fcwPsqlanl03','TMModelingCube')
query = 'SELECT NON EMPTY { [Measures].[Commission Net] } ON COLUMNS, NON EMPTY { ([Sold To].[Customer Name Number].[Customer Name Number].ALLMEMBERS ) } DIMENSION PROPERTIES MEMBER_CAPTION, MEMBER_UNIQUE_NAME ON ROWS FROM ( SELECT ( { [Publication Date].[Fiscal Year].&[2018], [Publication Date].[Fiscal Year].&[2019] } ) ON COLUMNS FROM ( SELECT ( { [Company].[Company].&[OSC], [Company].[Company].&[SSC] } ) ON COLUMNS FROM [Territory Management Modeling])) WHERE ( [Company].[Company].CurrentMember, [Publication Date].[Fiscal Year].CurrentMember ) CELL PROPERTIES VALUE, BACK_COLOR, FORE_COLOR, FORMATTED_VALUE, FORMAT_STRING, FONT_NAME, FONT_SIZE, FONT_FLAGS'
rs = win32com.client.Dispatch(r'ADODB.Recordset')
rs.Open(query, connString, CursorType=3)
a = rs.GetRows()
dataframe = pd.DataFrame(data= list(a)).transpose()
customer_list = list(dataframe[0].unique())

current_year = datetime.now().strftime('%Y')
last_year = str(int(datetime.now().strftime('%Y'))-1)

query_result = pd.DataFrame()
#iterate through customer names
for i in chunks(customer_list,500):
    z = ",".join(list(map(lambda x: '[Sold To].[Customer Name Number].&['+x+']',i)))
    query = 'SELECT NON EMPTY {{ [Measures].[Commission Net] }} ON COLUMNS, NON EMPTY {{ ([TM Sales Assignment].[SA ID].[SA ID].ALLMEMBERS * [TM Sales Assignment].[Sales Assignment Sub Team].[Sales Assignment Sub Team].ALLMEMBERS * [TM Sales Assignment].[Sales Assignment Employee].[Sales Assignment Employee].ALLMEMBERS * [Sold To].[Parent Name Number].[Parent Name Number].ALLMEMBERS * [Sold To].[Parent Name].[Parent Name].ALLMEMBERS * [Sold To].[Customer Name].[Customer Name].ALLMEMBERS * [Sold To].[Customer Name Number].[Customer Name Number].ALLMEMBERS * [Product].[Product Type].[Product Type].ALLMEMBERS * [Publication Date].[Fiscal Period].[Fiscal Period].ALLMEMBERS * [Publication Date].[Fiscal Quarter].[Fiscal Quarter].ALLMEMBERS * [Publication Date].[Date].[Date].ALLMEMBERS ) }} DIMENSION PROPERTIES MEMBER_CAPTION, MEMBER_UNIQUE_NAME ON ROWS FROM ( SELECT ( {{ [Publication Date].[Fiscal Year].&[2018], [Publication Date].[Fiscal Year].&[2019] }} ) ON COLUMNS FROM ( SELECT ( {{ [Company].[Company Code].&[SSC], [Company].[Company Code].&[OSC] }} ) ON COLUMNS FROM ( SELECT ( {{ {} }} ) ON COLUMNS FROM [Territory Management Modeling]))) WHERE ( [Company].[Company Code].CurrentMember, [Reporting Date].[Fiscal Year].CurrentMember ) CELL PROPERTIES VALUE, BACK_COLOR, FORE_COLOR, FORMATTED_VALUE, FORMAT_STRING, FONT_NAME, FONT_SIZE, FONT_FLAGS'.format(z)
    rs = win32com.client.Dispatch(r'ADODB.Recordset')
    rs.Open(query, connString, CursorType=3)
    a = rs.GetRows()
    query_result = query_result.append(pd.DataFrame(data= list(a)).transpose())
    
query_result.rename(columns = {0:'sa_id', 2:'sub_team', 4:'tm_employee_name', 6:'parent_name_number',
                     8:'parent_name', 10:'customer_name', 12:'customer_name_number', 14:'product_type',
                     16:'fiscal_period', 18:'fiscal_quarter', 20:'pub_date', 22:'commnet',},inplace=True)

query_result = query_result[['sa_id', 'sub_team', 'tm_employee_name', 'parent_name_number',
                     'parent_name', 'customer_name', 'customer_name_number', 'product_type',
                     'fiscal_period', 'fiscal_quarter', 'pub_date', 'commnet']]

query_result = query_result.merge(rep_mappings, how='inner', left_on = ['sa_id'], right_on  = ['sa_id'])
query_result.drop(labels = ['fiscal_year'], axis =1, inplace=True)
query_result.rename(columns = {'rep_name':'sales_rep_name'}, inplace=True)

In [42]:
query_result = query_result.merge(rep_mappings, how='inner', left_on = ['sa_id'], right_on  = ['sa_id'])

In [49]:
query_result.drop(labels = ['fiscal_year'], axis =1, inplace=True)
query_result.rename(columns = {'rep_name':'sales_rep_name'}, inplace=True)

In [50]:
query_result

,sa_id,sub_team,tm_employee_name,parent_name_number,parent_name,customer_name,customer_name_number,product_type,fiscal_period,fiscal_quarter,pub_date,commnet,company_name,sales_rep_name,manager_name,sales_team
0,SA00277,OSC_Media_2,"Cello,Curtis",50 Floor Inc (OSC)-CU00527832,50 Floor Inc (OSC),50 Floor Inc (OSC),50 Floor Inc (OSC)-CU00527832,Alternative Print,2018-06,2018-Q2,2018-06-14,1230,"Orlando Sentinel Communications Company, LLC",OPEN (Jahns),"Murphy, Carmena S",Media
1,SA00277,OSC_Media_2,"Cello,Curtis",50 Floor Inc (OSC)-CU00527832,50 Floor Inc (OSC),50 Floor Inc (OSC),50 Floor Inc (OSC)-CU00527832,Alternative Print,2018-06,2018-Q2,2018-06-14,1230,"Orlando Sentinel Communications Company, LLC",OPEN (Jahns),"Murphy, Carmena S",Media
2,SA00277,OSC_Media_2,"Cello,Curtis",50 Floor Inc (OSC)-CU00527832,50 Floor Inc (OSC),50 Floor Inc (OSC),50 Floor Inc (OSC)-CU00527832,Alternative Print,2018-06,2018-Q2,2018-06-21,1230,"Orlando Sentinel Communications Company, LLC",OPEN (Jahns),"Murphy, Carmena S",Media
3,SA00277,OSC_Media_2,"Cello,Curtis",50 Floor Inc (OSC)-CU00527832,50 Floor Inc (OSC),50 Floor Inc (OSC),50 Floor Inc (OSC)-CU00527832,Alternative Print,2018-06,2018-Q2,2018-06-21,1230,"Orlando Sentinel Communications Company, LLC",OPEN (Jahns),"Murphy, Carmena S",Media
4,SA00277,OSC_Media_2,"Cello,Curtis",50 Floor Inc (OSC)-CU00527832,50 Floor Inc (OSC),50 Floor Inc (OSC),50 Floor Inc (OSC)-CU00527832,Alternative Print,2018-06,2018-Q2,2018-06-28,1230,"Orlando Sentinel Communications Company, LLC",OPEN (Jahns),"Murphy, Carmena S",Media
5,SA00277,OSC_Media_2,"Cello,Curtis",50 Floor Inc (OSC)-CU00527832,50 Floor Inc (OSC),50 Floor Inc (OSC),50 Floor Inc (OSC)-CU00527832,Alternative Print,2018-06,2018-Q2,2018-06-28,1230,"Orlando Sentinel Communications Company, LLC",OPEN (Jahns),"Murphy, Carmena S",Media
6,SA00277,OSC_Media_2,"Cello,Curtis",50 Floor Inc (OSC)-CU00527832,50 Floor Inc (OSC),50 Floor Inc (OSC),50 Floor Inc (OSC)-CU00527832,Alternative Print,2018-07,2018-Q3,2018-07-05,1230,"Orlando Sentinel Communications Company, LLC",OPEN (Jahns),"Murphy, Carmena S",Media
7,SA00277,OSC_Media_2,"Cello,Curtis",50 Floor Inc (OSC)-CU00527832,50 Floor Inc (OSC),50 Floor Inc (OSC),50 Floor Inc (OSC)-CU00527832,Alternative Print,2018-07,2018-Q3,2018-07-05,1230,"Orlando Sentinel Communications Company, LLC",OPEN (Jahns),"Murphy, Carmena S",Media
8,SA00277,OSC_Media_2,"Cello,Curtis",50 Floor Inc (OSC)-CU00527832,50 Floor Inc (OSC),50 Floor Inc (OSC),50 Floor Inc (OSC)-CU00527832,Alternative Print,2018-07,2018-Q3,2018-07-12,1230,"Orlando Sentinel Communications Company, LLC",OPEN (Jahns),"Murphy, Carmena S",Media
9,SA00277,OSC_Media_2,"Cello,Curtis",50 Floor Inc (OSC)-CU00527832,50 Floor Inc (OSC),50 Floor Inc (OSC),50 Floor Inc (OSC)-CU00527832,Alternative Print,2018-07,2018-Q3,2018-07-12,1230,"Orlando Sentinel Communications Company, LLC",OPEN (Jahns),"Murphy, Carmena S",Media


In [11]:
z = list(map(lambda x: '[Sold To].[Customer Name Number].&['+x+']',customer_list))
z = ",".join(z)
# import ast
# z = ast.literal_eval(z)
# z = re.sub("^'", '', z)
# z = re.sub("'$", '', z)

In [12]:
z

'[Sold To].[Customer Name Number].&[ Andreasen Associates, Inc.-CU00650057],[Sold To].[Customer Name Number].&[ Yesmin Turner-CU00388915],[Sold To].[Customer Name Number].&[(C)ALBERT DAVID ABDOOL-CU00644526],[Sold To].[Customer Name Number].&[*DNU Brezniak-Rodman Chapel-CU00292150],[Sold To].[Customer Name Number].&[10 Spot Hair Studio-CU00184809],[Sold To].[Customer Name Number].&[10400 Atlantic  Inc-CU80003492],[Sold To].[Customer Name Number].&[1040005314-CU00638246],[Sold To].[Customer Name Number].&[1106 Legacy-CU80021519],[Sold To].[Customer Name Number].&[1200 Hillsboro Mile LLC-CU00212034],[Sold To].[Customer Name Number].&[1220 Design-CU00645320],[Sold To].[Customer Name Number].&[123college.com-CU00104185],[Sold To].[Customer Name Number].&[13th Floor Investments/Central Parc-CU00383368],[Sold To].[Customer Name Number].&[1428 East Semeron, LLC-CU00646153],[Sold To].[Customer Name Number].&[1500 Dental-CU00656310],[Sold To].[Customer Name Number].&[17th Judicial Circuit, Brow